In [3]:
import sys
import time
import numpy as np
import operator
import matplotlib.pyplot as plt
from sklearn import cross_validation
from datetime import datetime
from utils import *

%matplotlib inline

# Read climate sequence
- palce: hangzhou, zhejiang, China
- time: from 20120101 to 20151231
- features: morning rainfall, afternoon rainfall, night rainfall, min temperature, max temperature, wind force

In [5]:
climates = []
infile = open("../data/climate_hangzhou.csv","rb")
jump_header = False
for row in infile:
    if not jump_header:
        jump_header = True
        continue
    row = row.strip().decode("utf-8")
    items = row.split(",")
    dates = items[0]
    rainfall_morning = float(items[1])
    rainfall_afternoon = float(items[2])
    rainfall_night = float(items[3])
    min_temperature = float(items[4])
    max_temperature = float(items[5])
    wind_forece = float(items[6])
    climates.append([rainfall_afternoon,min_temperature,max_temperature,wind_forece])
#copy another four years datas
climates += climates
print "climates sequence length: "+str(len(climates))

climates sequence length: 2918


# Read patient visit records
- records come from a hangzhou hospital
- each row contains patient id, diagnose code(icd9), diagnose time (for example: 0,R10.4,2013-01-15 11:32:14)

In [6]:
patient_to_records = {}
disease_set = set()
disease_to_frequency = {}
infile = open("../data/patient_visit_record.csv","rb")
for row in infile:    
    row = row.strip().decode('utf-8')
    items = row.split(',')
    if len(items) != 3: continue
    pid = int(items[0])
    disease = items[1]
    visit_date_time = items[2]
    #unknown data format exception
    if u"0001-01-01" in visit_date_time:
        continue
    visit_date = visit_date_time.split()[0]
    visit_time = visit_date_time.split()[1]
    visit_date_format = time.strptime(visit_date, "%Y-%m-%d")
    patient_to_records.setdefault(pid,{}).setdefault(visit_date_format,[]).append(disease)
    disease_set.add(disease)
    disease_to_frequency.setdefault(disease,0)
    disease_to_frequency[disease] += 1
print "patient size: "+str(len(patient_to_records.keys()))

patient size: 141007


# Create index for disease

In [7]:
disease_to_index = {}
index_to_disease = {}
disease_index = 0
for disease in disease_set:
    disease_to_index[disease] = disease_index
    index_to_disease[disease_index] = disease
    disease_index += 1
disease_size = len(disease_to_index.keys())
print "disease size: "+str(disease_size)
disease_frequency_sorted = sorted(disease_to_frequency.items(),key=lambda p:p[1],reverse=True)
for t in disease_frequency_sorted[:5]:
    print t[0]+" "+str(t[1])

disease size: 4006
R10.4 22864
K29.7 22462
I10.x 21398
R05.x 19241
J06.9 18573


# Create datas and targets

In [9]:
datas = []
targets = []
MIN_VISIT_NUM = 5
#one interval_unit represents k days, here we define k = 7 which is a week
interval_unit = 7 
for patient in patient_to_records.keys():
    records = []
    records_sorted = sorted(patient_to_records[patient].items(),key=lambda p:p[0],reverse=False)
    pre_visit_date = None
    for record in records_sorted:
        visit_date = record[0]
        visit_day_index = (visit_date[0] - 2012)*365 + (visit_date[7])
        diseases = record[1]
        disease_index_list = [disease_to_index[disease] for disease in diseases] 
        visit_index_unitize = int(visit_day_index / interval_unit)
        records.append([visit_index_unitize]+disease_index_list)
        pre_visit_date = visit_date
    if len(records) < MIN_VISIT_NUM: continue
    datas.append(records[:-1])
    targets.append(records[1:])

# Create train and test

In [10]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(datas, targets, test_size=0.1, random_state=0)
print "train size: "+str(len(X_train))
print "test size: "+str(len(X_test))

train size: 79916
test size: 8880


# Recurrent Neural Network Implementation

In [11]:
#define RNN
class RNNNumpy:
    def __init__(self, feature_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.input_dim = feature_dim
        self.output_dim = feature_dim
        self.hidden_dim = hidden_dim
        #4 feature each day and consider the current week and previous week, totally 7*2 days
        self.climate_input_dim = 4*7*2
        self.climate_hidden_dim = 10
        self.bptt_truncate = bptt_truncate
        self.predict_n_unit = 52
        #predict top k diseases for a patient
        self.k = 10
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./self.input_dim), np.sqrt(1./self.input_dim), (hidden_dim, self.input_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))
        self.Q = np.random.uniform(-np.sqrt(1./self.climate_input_dim), np.sqrt(1./self.climate_input_dim), (self.climate_hidden_dim, self.climate_input_dim))
        self.T = np.random.uniform(-np.sqrt(1./self.climate_hidden_dim), np.sqrt(1./self.climate_hidden_dim), (hidden_dim, self.climate_hidden_dim))
        self.P = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (self.output_dim, hidden_dim))

In [12]:
#forward propagation for one sample
def forward_propagation(self, x, climates):
    T = len(x)
    s = np.zeros((T+1, self.hidden_dim))
    s[-1] = np.zeros(self.hidden_dim)
    c = np.zeros((T, self.predict_n_unit, self.climate_input_dim))
    h = np.zeros((T, self.predict_n_unit, self.climate_hidden_dim))
    m = np.zeros((T, self.predict_n_unit, self.hidden_dim))
    o = np.zeros((T, self.predict_n_unit, self.output_dim))
    for t in np.arange(T):
        visit_index_unitize = int(x[t][0])
        x_input = np.zeros(self.input_dim)
        x_input[x[t][1:]] = 1
        s[t] = np.tanh(self.U.dot(x_input) + self.W.dot(s[t-1]))
        for future_visit_offset in range(self.predict_n_unit):   
            c[t][future_visit_offset] = np.array([climates[(visit_index_unitize + future_visit_offset + 1) * 7 - i] for i in range(14)]).ravel()
            h[t][future_visit_offset] = np.tanh(self.Q.dot(c[t][future_visit_offset]))
            m[t][future_visit_offset] = np.tanh(self.V.dot(s[t]) + self.T.dot(h[t][future_visit_offset]))
            o[t][future_visit_offset] = softmax(self.P.dot(m[t][future_visit_offset]))
    return [o, m, h, c, s]

RNNNumpy.forward_propagation = forward_propagation

In [13]:
def calculate_total_loss(self, X, Y):
    Loss = 0
    # For each sample
    for i in np.arange(len(Y)):
        o, m, h, c, s = self.forward_propagation(X[i],climates)
        for t in range(len(Y[i])):
            current_visit_index_unitize = int(X[i][t][0])
            future_visit_index_unitize = int(Y[i][t][0])
            future_visit_offset = future_visit_index_unitize - current_visit_index_unitize
            future_diseases = Y[i][t][1:]
            if future_visit_offset >= self.predict_n_unit:
                future_visit_offset = self.predict_n_unit - 1
            correct_predict_probability = o[t,future_visit_offset,future_diseases]
            Loss += -1 * np.sum(np.log(correct_predict_probability))
    return Loss

def calculate_loss(self, X, Y):
    # Divide the total loss by the number of training examples
    N = np.sum((len(y_i) for y_i in Y))
    return self.calculate_total_loss(X,Y)/N

RNNNumpy.calculate_total_loss = calculate_total_loss
RNNNumpy.calculate_loss = calculate_loss

In [14]:
model = RNNNumpy(disease_size)
print "Loss: %f" % model.calculate_loss(X_train[:100], y_train[:100])

Loss: 10.246454


In [15]:
def bptt(self, x, y):
    T = len(y)
    # Perform forward propagation
    o, m, h, c, s = self.forward_propagation(x,climates)
    # We accumulate the gradients in these variables
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    dLdQ = np.zeros(self.Q.shape)
    dLdT = np.zeros(self.T.shape)
    dLdP = np.zeros(self.P.shape)
    
    delta_o = o
    for t in range(T):
        y_sum = len(y[t][1:])
        current_visit_index_unitize = int(x[t][0])
        future_visit_index_unitize = int(y[t][0])
        future_visit_offset = future_visit_index_unitize - current_visit_index_unitize
        future_diseases = y[t][1:]
        if future_visit_offset >= self.predict_n_unit:
            future_visit_offset = self.predict_n_unit - 1
        delta_o[t,future_visit_offset,np.arange(self.output_dim)] *= y_sum
        delta_o[t,future_visit_offset,future_diseases] -= 1.

    # For each output backwards...
    for t in np.arange(T)[::-1]:
        current_visit_index_unitize = int(x[t][0])
        future_visit_index_unitize = int(y[t][0])
        future_visit_offset = future_visit_index_unitize - current_visit_index_unitize
        if future_visit_offset >= self.predict_n_unit: future_visit_offset = self.predict_n_unit - 1
        dLdP += np.outer(delta_o[t][future_visit_offset],m[t][future_visit_offset])   
        delta_m = self.P.T.dot(delta_o[t][future_visit_offset]) * (1 - (m[t][future_visit_offset] ** 2))
        dLdT += np.outer(delta_m, h[t][future_visit_offset])
        dLdV += np.outer(delta_m, s[t])
        delta_c = self.T.T.dot(delta_m) * (1 - (h[t][future_visit_offset] ** 2))
        dLdQ += np.outer(delta_c,c[t][future_visit_offset])
        # Initial delta calculation
        delta_t = self.V.T.dot(delta_m) * (1 - (s[t] ** 2))
        # Backpropagation through time (for at most self.bptt_truncate steps)
        for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
            # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
            dLdW += np.outer(delta_t, s[bptt_step-1])
            X = np.zeros(self.input_dim)
            X[x[bptt_step][1:]] = 1            
            dLdU += np.outer(delta_t, X)
            # Update delta for next step
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
    return [dLdP, dLdT, dLdQ, dLdV, dLdU, dLdW]

RNNNumpy.bptt = bptt

In [16]:
def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
    # Calculate the gradients using backpropagation. We want to checker if these are correct.
    bptt_gradients = model.bptt(x, y)
    # List of all parameters we want to check.
    model_parameters = ['P','T','Q','V','U','W']
    # Gradient check for each parameter
    for pidx, pname in enumerate(model_parameters):
        # Get the actual parameter value from the mode, e.g. model.W
        parameter = operator.attrgetter(pname)(self)
        print parameter.shape
        print "Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape))
        # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
        it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            ix = it.multi_index
            # Save the original value so we can reset it later
            original_value = parameter[ix]
            # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
            parameter[ix] = original_value + h
            gradplus = model.calculate_total_loss([x],[y])
            parameter[ix] = original_value - h
            gradminus = model.calculate_total_loss([x],[y])
            estimated_gradient = (gradplus - gradminus)/(2*h)
            # Reset parameter to original value
            parameter[ix] = original_value
            # The gradient for this parameter calculated using backpropagation
            backprop_gradient = bptt_gradients[pidx][ix]
            # calculate The relative error: (|x - y|/(|x| + |y|))
            relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
            # If the error is to large fail the gradient check
            if relative_error > error_threshold:
                print "Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix)
                print "+h Loss: %f" % gradplus
                print "-h Loss: %f" % gradminus
                print "Estimated_gradient: %f" % estimated_gradient
                print "Backpropagation gradient: %f" % backprop_gradient
                print "Relative Error: %f" % relative_error
                return 
            it.iternext()
        print "Gradient check for parameter %s passed." % (pname)

RNNNumpy.gradient_check = gradient_check

# To avoid performing millions of expensive calculations we use a smaller vocabulary size for checking.
grad_check_vocab_size = 100
np.random.seed(10)
model = RNNNumpy(grad_check_vocab_size, 10, bptt_truncate=1000)
model.gradient_check([[0,1,2],[3,4,5],[6,7,8]], [[3,4,5],[6,7,8],[9,10,11]])

(100, 10)
Performing gradient check for parameter P with size 1000.
Gradient check for parameter P passed.
(10, 10)
Performing gradient check for parameter T with size 100.
Gradient check for parameter T passed.
(10, 56)
Performing gradient check for parameter Q with size 560.
Gradient check for parameter Q passed.
(10, 10)
Performing gradient check for parameter V with size 100.
Gradient check for parameter V passed.
(10, 100)
Performing gradient check for parameter U with size 1000.
Gradient check for parameter U passed.
(10, 10)
Performing gradient check for parameter W with size 100.
Gradient check for parameter W passed.


/Library/Python/2.7/site-packages/ipykernel/__main__.py:29: RuntimeWarning: invalid value encountered in double_scalars


In [17]:
# Performs one step of SGD.
def numpy_sdg_step(self, x, y, learning_rate):
    # Calculate the gradients
    dLdP, dLdT, dLdQ, dLdV, dLdU, dLdW = self.bptt(x, y)
    # Change parameters according to gradients and learning rate
    self.P -= learning_rate * dLdP
    self.T -= learning_rate * dLdT
    self.Q -= learning_rate * dLdQ
    self.U -= learning_rate * dLdU
    self.V -= learning_rate * dLdV
    self.W -= learning_rate * dLdW

RNNNumpy.sgd_step = numpy_sdg_step

In [18]:
# Outer SGD Loop
# - model: The RNN model instance
# - X_train: The training data set
# - y_train: The training data labels
# - learning_rate: Initial learning rate for SGD
# - nepoch: Number of times to iterate through the complete dataset
# - evaluate_loss_after: Evaluate the loss after this many epochs
def train_with_sgd(model, X_train, y_train, learning_rate=0.05, nepoch=10, evaluate_loss_after=5):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        # Optionally evaluate the loss
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print "%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss)
            print "map: " + str(meanAveragePrecision(model, X_test[:100], y_test[:100], disease_size))
            print "ar: "+str(averageRank(model,X_test[:100],y_test[:100],disease_size))
            # Adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5  
                print "Setting learning rate to %f" % learning_rate
            sys.stdout.flush()
        # For each training example...
        for i in range(len(y_train)):
            # One SGD step
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1

In [ ]:
np.random.seed(10)
model = RNNNumpy(disease_size)
losses = train_with_sgd(model, X_train[:1000], y_train[:1000], nepoch=100, evaluate_loss_after=1)

# Evaluation Metric
- AR(Average Rank)
- MAP(Mean Average Precision)

In [2]:
def averageRank(model, X_test, y_test, top_k=10):
    average_rank = 0
    counter = 0
    for i in range(len(X_test)):
#         if (i % 2000 == 0): print "current handling at: "+str(i)
        x = X_test[i]
        y = y_test[i]
        pre_visit_index_unitize = x[-1][0]
        last_visit_index_unitize = y[-1][0]
        last_diseases = y[-1][1:]
        o, m, h, c, s = model.forward_propagation(x,climates)
        last_visit_offset = last_visit_index_unitize - pre_visit_index_unitize
        if last_visit_offset >= model.predict_n_unit:
            last_visit_offset = model.predict_n_unit - 1
        predict = list(o[-1][last_visit_offset].argsort()[-top_k:][::-1])
        predict_baseline = [disease_to_index[val[0]] for val in disease_frequency_sorted]
        for disease_index in last_diseases:
            average_rank += predict.index(disease_index)+1
            counter += 1
    return 1.0*average_rank/counter      

In [3]:
def meanAveragePrecision(model, X_test, y_test, top_k=10):
    mean_average_precision = 0
    for i in range(len(X_test)):
#         if (i % 2000 == 0): print "current handling at: "+str(i)
        x = X_test[i]
        y = y_test[i]
        pre_visit_index_unitize = x[-1][0]
        last_visit_index_unitize = y[-1][0]
        last_diseases = y[-1][1:]
        o, m, h, c, s = model.forward_propagation(x,climates)
        last_visit_offset = last_visit_index_unitize - pre_visit_index_unitize
        if last_visit_offset >= model.predict_n_unit:
            last_visit_offset = model.predict_n_unit - 1
        predict = list(o[-1][last_visit_offset].argsort()[-top_k:][::-1])
        predict_baseline = [disease_to_index[val[0]] for val in disease_frequency_sorted]
        ranks = []
        mean_average_precision_item = 0
        for disease_index in last_diseases:
            ranks.append(predict.index(disease_index)+1)
        ranks_sorted = sorted(ranks,reverse=False)
#         print ranks_sorted
        for i in range(len(ranks_sorted)):
            mean_average_precision_item += 1.0*(i+1)/ranks_sorted[i]
        mean_average_precision_item /= len(ranks_sorted)
#         print mean_average_precision_item
        mean_average_precision += mean_average_precision_item
    return 1.0*mean_average_precision/len(X_test)     

In [ ]:
print meanAveragePrecision(model,X_test,y_test,disease_size)
print averageRank(model,X_test,y_test,disease_size) 

# Experiment Result

In [ ]:
#train size = 1000, test size = 13967, iter = 10, iter interval = 4 mins, hidden = 100, ar = 261, map = 0.064
#train size = 5000, test size = 13967, iter = 10, iter interval = 30 mins, hidden = 100, ar = 198, map = 0.0667